In [1]:
from PIL import Image
from keras.objectives import binary_crossentropy, mean_squared_error
import numpy as np
from functools import reduce
from keras.engine.topology import Input, Network
from keras.engine.training import Model
#from keras.models import to_list
np.random.seed(1337)  # for reproducibility
from keras.utils.generic_utils import to_list
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.datasets import mnist
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Flatten, Dropout,GlobalAveragePooling2D
from keras.models import Model
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import pandas as pd
from PIL import Image

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
batch_size = 16
num_classes = 10
epochs = 100

In [0]:
train=np.load("/content/drive/My Drive/256_npy/picture.npy")
#train_label=np.load("/mnt/Doc/reserch1/train_type.npy")
##test=np.load("/mnt/Doc/512pix_to_128pix/npy/128pix_testpicture.npy")
val = np.load("/content/drive/My Drive/256_npy/picture_val.npy")
#test_label=np.load("/mnt/Doc/reserch1/test_type.npy")

#x_train = train.astype('float32')
#x_test = test.astype('float32')
#x_val = val.astype('float32')
train /= 255
#x_test /= 255
val /=255
#del train
#del test
#del val

In [0]:
SAMPLE_SIZE = 0

batch_size = 16
nb_classes = 10
nb_epoch = 1

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

In [0]:
def main(data, use_dropout, use_vat):
    np.random.seed(1337)  # for reproducibility

    # the data, shuffled and split between train and test sets
    #(X_train, y_train), (X_test, y_test) = data

    
    #X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    #X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    #input_shape = (img_rows, img_cols, 1)

    X_train = train.astype('float32')
    X_test = val.astype('float32')
    X_train /= 255.
    X_test /= 255.

    # convert class vectors to binary class matrices
    #y_train = np_utils.to_categorical(y_train, nb_classes)
    #y_test = np_utils.to_categorical(y_test, nb_classes)

    #if SAMPLE_SIZE:
    #   X_train = X_train[:SAMPLE_SIZE]
    #   y_train = y_train[:SAMPLE_SIZE]
    #   X_test = X_test[:SAMPLE_SIZE]
    #   y_test = y_test[:SAMPLE_SIZE]
    
    input_layer =(256,256,3)
    print("start: use_dropout=%s, use_vat=%s" % (use_dropout, use_vat))
    my_model = MyModel(input_layer, use_dropout, use_vat).build()
    my_model1 = my_model.save_best("/content/drive/My Drive/256_npy/best_weight.h5")
    my_model1.training(X_train, X_train, X_test, X_test,epoch=13)
    my_model1.weight_save("/content/drive/My Drive/research_/weight/wight_vat1.h5")
    #score = my_model.model.evaluate(X_test, y_test, verbose=0)
    #print("finish: use_dropout=%s, use_vat=%s: score=%s, accuracy=%s" % (use_dropout, use_vat, score[0], score[1]))


In [0]:
class MyModel:
    model = None

    def __init__(self, input_shape, use_dropout=True, use_vat=True):
        self.input_shape = input_shape
        self.use_dropout = use_dropout
        self.use_vat = use_vat

    def build(self):
        input_layer = Input(self.input_shape)
        output_layer = self.core_data_flow(input_layer)
        if self.use_vat:
            self.model = VATModel(input_layer, output_layer).setup_vat_loss()
        else:
            self.model = Model(input_layer, output_layer)
        return self
    
    def  save_best(self,path_):
        self.modelCheckpoint = ModelCheckpoint(filepath =path_,
                                 monitor="val_loss",
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode="min",
                                 period=1)
        return self
    
    def core_data_flow(self, input_layer):
        x = Conv2D(4, (7, 7), padding='same')(input_layer)
        #x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(8, (5, 5), padding='same')(x)
        #x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(16, (3, 3), padding='same')(x)
        #x = BatchNormalization()(x)
        x = Activation('relu')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(16, (3, 3), padding='same')(encoded)
        #x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(8, (5, 5), padding='same')(x)
        #x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(4, (7, 7), padding='same')(x)
        #x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(3, (3, 3), padding='same')(x)
        #x = BatchNormalization()(x)
        decoded = Activation('relu')(x)
        
        return decoded
    def weight_save(self,path):
        self.model.save_weights(path)
    
   
        
    
    def training(self, X_train, y_train, X_test, y_test,epoch):
        self.model.compile(loss="mean_squared_error", optimizer='adam', metrics=['accuracy'])
        np.random.seed(1337)  # for reproducibility
        self.model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epoch,
                       verbose=1, validation_data=(X_test, y_test),callbacks=[self.modelCheckpoint])
     


class VATModel(Model):
    _vat_loss = None

    def setup_vat_loss(self, eps=1, xi=10, ip=1):
        self._vat_loss = self.vat_loss(eps, xi, ip)
        return self


    def vat_loss(self, eps, xi, ip):
        normal_outputs = [K.stop_gradient(x) for x in to_list(self.outputs)]
        d_list = [K.random_normal(K.shape(x)) for x in self.inputs]
        print(d_list)

        for _ in range(ip):
            new_inputs = [x + self.normalize_vector(d)*xi for (x, d) in zip(self.inputs, d_list)]
            new_outputs = to_list(self.call(new_inputs))
            klds = [K.sum(self.kld(normal, new)) for normal, new in zip(normal_outputs, new_outputs)]
            kld = reduce(lambda t, x: t+x, klds, 0)
            d_list = [K.stop_gradient(d) for d in K.gradients(kld, d_list)]

        new_inputs = [x + self.normalize_vector(d) * eps for (x, d) in zip(self.inputs, d_list)]
        y_perturbations = to_list(self.call(new_inputs))
        klds = [K.mean(self.kld(normal, new)) for normal, new in zip(normal_outputs, y_perturbations)]
        kld = reduce(lambda t, x: t + x, klds, 0)
        return kld

    @staticmethod
    def normalize_vector(x):
        z = K.sum(K.batch_flatten(K.square(x)), axis=1)
        while K.ndim(z) < K.ndim(x):
            z = K.expand_dims(z, axis=-1)
        return x / (K.sqrt(z) + K.epsilon())

    @staticmethod
    def kld(p, q):
        v = p * (K.log(p + K.epsilon()) - K.log(q + K.epsilon()))
        return K.sum(K.batch_flatten(v), axis=1, keepdims=True)


data = mnist.load_data()
#main(data, use_dropout=False, use_vat=False)
#main(data, use_dropout=True, use_vat=False)
#main(data, use_dropout=False, use_vat=True)
main(data, use_dropout=True, use_vat=True)

In [0]:
.modell.save_weight("/content/drive/My Drive/research_/weight/wight_vat1.h5")


In [0]:
my_model

In [0]:
score = my_model.model.evaluate(X_test, X_test, verbose=0)

In [0]:
#print('x_train shape:', x_train.shape)
print(train.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')
print(val.shape[0], 'val samples')


In [0]:
# divide x_test into validation and test
#x_val = x_test[:900]
#x_test = x_test[900:]
#x_val_label = test_label[:900]
#x_test_label = test_label[900:]

In [0]:
#train_label_1=np_utils.to_categorical(train_label)
#test_label_1=np_utils.to_categorical(x_test_label)
#val_label_1=np_utils.to_categorical(x_val_label)


In [0]:
#print("validation data: {0} \ntest data: {1}".format(x_val.shape, x_test.shape))

In [0]:
input_img = Input(shape=(256, 256, 3))
x = Conv2D(4, (7, 7), padding='same')(input_img)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(encoded)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (5, 5), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(4, (7, 7), padding='same')(x)
#x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
#x = BatchNormalization()(x)
decoded = Activation('relu')(x)

In [0]:
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [0]:
modelCheckpoint = ModelCheckpoint(filepath ="/mnt/Doc/weight/Autoencoder/256/best_weight_0507.h5",
                                 monitor="val_loss",
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode="min",
                                 period=1)

In [0]:
history = model.fit(train, train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(val, val),
                    callbacks=[modelCheckpoint],
                    shuffle=True)

In [0]:
from datetime import datetime
x = datetime.now().strftime("%Y.%m.%d %H:%M:%S")
y = datetime.now().strftime("%Y.%m.%d")
new_file_pass = os.path.join("/mnt/Doc/Autoencoder_data",y)
if os.path.exists(new_file_pass)==False:
    os.mkdir(new_file_pass)
else:
    pass
y1 = os.path.join(new_file_pass,"weight"+x+".h5")
z1 = os.path.join(new_file_pass,"model"+x+".json")
model.save_weights(y)
open(z1,"w").write(model.to_json())

In [0]:
open("autoencodermodel1.json","w").write(model.to_json())

In [0]:
model.load_weights('/mnt/Doc/weight/Autoencoder/256/best_weight_0426.h5')

In [0]:
images = []


In [0]:
images = []
path = "/mnt/Doc/2048pix_tissue_all/TRU/TCGA-50-5935-01A-01-TS1_2048_files/8_15.jpeg"
path = "/mnt/Doc/2048pix_tissue_all/PP/TCGA-78-7160-01A-01-TS1_2048_files/6_5.jpeg"
f = Image.open(path) 
images.append((np.asarray(f.convert('RGB'), dtype=np.float32))/255)
del f
f1 = Image.open(path) 
images.append((np.asarray(f1.convert('RGB'), dtype=np.float32))/255)
del f1

In [0]:
decoded_imgs=model.predict(images)

In [0]:
import matplotlib.pyplot as plt
import random

In [0]:
decoded_imgs.astype('uint8')
plt.imshow(decoded_imgs[9])

In [0]:
plt.imshow(x_test[2])

In [0]:
n=10
plt.figure(figsize=(20,4))
list1=range(len(val))
randam1=random.sample(list1,10)
for i in range(n):
    #オリジナル画像
    picture_num=randam1[i]
    ax=plt.subplot(2,n,i+1)
    plt.imshow(val[picture_num])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    #変換後画像
    ax=plt.subplot(2,n,i+1+n)
    plt.imshow(decoded_imgs[picture_num])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    

In [0]:
model.summary()

In [0]:
input_img = Input(shape=(128, 128, 3))
x = Conv2D(32, (7, 7), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (5, 5), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(encoded)
x = Dense(128)(x)
x = Activation("relu")(x)
x = Dense(3)(x)
out = Activation("softmax")

In [0]:
model = Model(input_img, out)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

In [0]:
import os 
import h5py
from keras import backend as K


In [0]:
weights_path = ''

assert os.path.exists(weights_path), 'Model weights not found (see "weights_path" variable in script).'
f = h5py.File(weights_path)
layer_names = [n.decode('utf8') for n in f.attrs['layer_names']]
weight_value_tuples = []
for k, name in enumerate(layer_names):
    if k >= len(model.layers):
        # 全結合層の重みは読み込まない
        break
    g = f[name]
    weight_names = [n.decode('utf8') for n in g.attrs['weight_names']]
    if len(weight_names):
        weight_values = [g[weight_name] for weight_name in weight_names]
        layer = model.layers[k]
        symbolic_weights = layer.trainable_weights + layer.non_trainable_weights
        if len(weight_values) != len(symbolic_weights):
            raise Exception('Layer #' + str(k) +
                            ' (named "' + layer.name +
                            '" in the current model) was found to '
                            'correspond to layer ' + name +
                            ' in the save file. '
                            'However the new layer ' + layer.name +
                            ' expects ' + str(len(symbolic_weights)) +
                            ' weights, but the saved weights have ' +
                            str(len(weight_values)) +
                            ' elements.')
        weight_value_tuples += zip(symbolic_weights, weight_values)
K.batch_set_value(weight_value_tuples)
f.close()
print('Model loaded.')

In [0]:
layer_names

In [0]:
g = f["conv2d_15"]
weight_names = [n.decode("utf8") for n in g.attrs["weight_names"]]

In [0]:
#メモリ使用量確認
def print_varsize():
    import types
    print("{}{: >15}{}{: >10}{}".format('|','Variable Name','|','  Size','|'))
    print(" -------------------------- ")
    for k, v in globals().items():
        if hasattr(v, 'size') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(v.size),'|'))
        elif hasattr(v, '__len__') and not k.startswith('_') and not isinstance(v,types.ModuleType):
            print("{}{: >15}{}{: >10}{}".format('|',k,'|',str(len(v)),'|'))

In [0]:
print_varsize()

In [0]:
del test_label_1

In [0]:
x_val